## Clip a Wflow model

Let's imagine you have built and calibrated a **Wflow** model for a whole country or continent. And now you have a new project for a small basin belonging to your big calibrated Wflow model. Instead of re-building the smaller model fomr scratch with HydroMT and re-doing the calibration steps, you can easily use the **clip** method of the **WflowModel** class from HydroMT-Wflow to directly extract your smaller catchment out of your already existing big calibrated model!

This notebook demonstrates how to clip a **Wflow** model from an already existing one using python.

### Clip a small sub-basin

In [ ]:
import numpy as np
from hydromt_wflow import WflowModel

In [ ]:
# Read the original large model
full_model = WflowModel(root="wflow_piave_subbasin", mode="r")
full_model.read()

In [ ]:
# Clip the model using a new region and write the result to a new path
output_dir = "wflow_piave_clip"

# Derive a subbasin with its outlet at the given x,y coordinates (WGS84) snapped to a river with minimum stream order (wflow_streamorder) of 4. 
# All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/model_region)
# NOTE: Compared to build, you may notice here that the streamorder argument is called 'wflow_streamorder'.
# As we are clipping a wflow model, the name here should correspond to the name of the stream order map inside of your wflow model.
region = {
    "subbasin": [12.3006, 46.4324],
    'wflow_streamorder': 4
}

full_model.set_root(root=output_dir, mode="w")
full_model.clip(region=region)
full_model.write()

In [ ]:
# Read in the clipped model
clipped_model = WflowModel(root=output_dir, mode="r")
clipped_model.read()

Compared to build, the clip command line has less options (no configuration file and no data catalog or resolution). Clip simply re-uses the original model data and extract all the informations from the model for our clipped region location.

### Visualize and/or inspect model schematization

The **wflow plot** example notebook contains scripts to visualize your model.

Here we will just simply plot the region of the different model (original and clipped) to check the differences between them.

In [ ]:
# Import plot packages
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt

# import descartes  # used under the hood to plot polygons

In [ ]:
# read/derive river geometries
gdf_riv_full = full_model.rivers
gdf_riv_clipped = clipped_model.rivers

# read/derive model basin boundary
gdf_bas_full = full_model.basins
gdf_bas_clipped = clipped_model.basins

In [ ]:
# we assume the model maps are in the geographic CRS EPSG:4326
proj = ccrs.PlateCarree()
# adjust zoomlevel and figure size to your basis size & aspect
zoom_level = 10
figsize = (10, 8)
shaded = False


# initialize image with geoaxes
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(projection=proj)
extent = np.array(full_model.grid.raster.box.buffer(0.02).total_bounds)[[0, 2, 1, 3]]
ax.set_extent(extent, crs=proj)

# add sat background image
ax.add_image(cimgt.QuadtreeTiles(), zoom_level, alpha=0.5)

# plot rivers with increasing width with stream order
gdf_riv_full.plot(ax=ax, lw=gdf_riv_full["strord"] / 2, color="blue", zorder=3, label="river")
gdf_riv_clipped.plot(
    ax=ax, lw=gdf_riv_clipped["strord"] / 2, color="purple", zorder=3, label="river clip"
)
# plot the basin boundary
gdf_bas_full.boundary.plot(ax=ax, color="k", linewidth=0.8, label="basin original")
gdf_bas_clipped.boundary.plot(ax=ax, color="r", linewidth=0.8, label="basin clip")

ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
ax.set_ylabel(f"latitude [degree north]")
ax.set_xlabel(f"longitude [degree east]")
_ = ax.set_title(f"wflow base map")
legend = ax.legend(
    handles=[*ax.get_legend_handles_labels()[0]],
    title="Legend",
    loc="lower right",
    frameon=True,
    framealpha=0.7,
    edgecolor="k",
    facecolor="white",
)